# Project : Computer Vision Anomally Detecting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install timm

     |████████████████████████████████| 431 kB 34.7 MB/s 


## Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

## Data load

In [2]:
path = './'

In [5]:
train_png = sorted(glob(path + 'data/train/*.png'))
test_png = sorted(glob(path + 'data/test/*.png'))

glob : 많은 파일들을 다뤄야 하는 파이썬 프로그램을 작성할 때, 특정한 패턴이나 확장자를 가진 파일들의 경로나 이름이 필요할 때가 있다. glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다.

In [6]:
len(train_png), len(test_png)

In [3]:
# class 속성에 필요한 label을 수집하는 부분
# df의 label 과 class를 추출하였다.

train_y = pd.read_csv(path +"data/train_df.csv")

train_labels = train_y["label"]
train_classes = train_y['class']

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}
class_unique = sorted(np.unique(train_classes))
class_unique = {key:value for key,value in zip(class_unique, range(len(class_unique)))}

train_labels = [label_unique[k] for k in train_labels]
train_classes = [class_unique[k] for k in train_classes]

In [4]:
# img resize

def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [ ]:
# glob으로 만들었던 이미지 file 이름 list를 사용하여 load하고 리스트를 만든다.

train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|███████████████████████████████████████| 2154/2154 [01:02<00:00, 34.61it/s]


In [ ]:
# 위에서 만든 이미지 리스트를 npy 형식으로 저장한다.

np.save(path + 'data/train_imgs_384', np.array(train_imgs))
np.save(path + 'data/test_imgs_384', np.array(test_imgs))

In [4]:
# 저장한 이미지를 불러온다

train_imgs = np.load(path + 'data/train_imgs_384.npy')
test_imgs = np.load(path + 'data/test_imgs_384.npy')

In [10]:
# RGB 평균((384, 384, 3) => (3)) 리스트
# 리스트의 각 RGB별 평균을 내서 사용

meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

# train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
# train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488

train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488


In [11]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

# test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
# test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663

test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663


☆ Custom_dataset : dataset 생성하는 class  
- \_\_init\_\_ : 생성자, 매개변수를 객체에 전달
- \_\_len\_\_ : img_paths 변수의 길이 return
- \_\_getitem\_\_ : 
    인스턴스 변수에 직접 접근하지 않고 객체 자체를 통해서 슬라이싱을 구현, idx를 받아서 return  
    img_paths 에서 idx 위치의 이미지를 transform하여 idx 번째의 label과 함께 출력  
    transform 시 test 이미지에서 Affine, Flip, Rotation 의 변형을 주었으며, metal_nut의 경우 Flip을 제외함
             
☆ Network : 
- timm모듈을 사용, 신경망 모델 정의  
- torch.nn.Module을 상속
- \_\_init\_\_ : 모델에서 사용될 module(efficient b4)정의 및 초기화
- forward() : 모델에서 실행되어야하는 계산을 정의 (output 정의), init에서 정의된것들을 연결하는 메소드  

☆ score_function   
☆ main : seed 정의   

In [5]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, classes, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.classes=classes
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
            if self.classes[idx]==7:
                # print(idx, '7')
                train_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                            std = [0.181572, 0.174035, 0.163234]),
                        transforms.RandomAffine((-45, 45)),
                    
                        transforms.RandomRotation((0,80))       #  이미지를 랜덤으로 degrees 각도로 회전한다.
                        
                    ])
                img = train_transform(img)
            else :
                train_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                            std = [0.181572, 0.174035, 0.163234]),
                        transforms.RandomAffine((-45, 45)),
                    
                        transforms.RandomVerticalFlip(p=0.5),   # - 이미지를 랜덤으로 수직으로 뒤집는다. p =0이면 뒤집지 않는다.
                        transforms.RandomHorizontalFlip(p=0.5), # - 이미지를 랜덤으로 수평으로 뒤집는다.
                        transforms.RandomRotation((0,80))       #  이미지를 랜덤으로 degrees 각도로 회전한다.
                        
                    ])
                img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323]),
              
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [6]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [7]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [15]:
train_y.groupby('label')['label'].count().ravel()

array([ 10,  11,  11, 209,   7,   6,   6,   7,   5, 224,   6,   5,   5,
        12,  11, 219,  11,  12,  10,  10,   9, 280,   9,   9,  10,   6,
         6,   6, 264,   6,   6,   9,   9, 391,   9,   9,  10,  10,   9,
        10, 245,   9,  13,  11,  12, 220,  12,  13,   9,  11,  13,  10,
       267,   5,  12, 320,  12,  12,  13,  12,  12,   9,   9, 230,   8,
         9,   8,  15,  60,   5,   5,   5, 213,   5,   4,   6, 247,   5,
         5,  11,  10,   8,   9,   8, 240,   9,   9,   8], dtype=int64)

In [19]:
856+3421

4277

In [28]:
import gc
# pd.set_option('display.max_row', 500)

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 32
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
    print("----------fold_{} start!----------".format(idx))
    print(train_idx)
    print(len(train_imgs))
    print(len(train_imgs[train_idx]))
    print(len(train_imgs[val_idx]))
    label_train = train_y['label'].iloc[train_idx]
    label_valid = train_y['label'].iloc[val_idx]
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_valid.value_counts())

----------fold_0 start!----------
[   0    1    2 ... 4274 4275 4276]
4277
3421
856
학습 레이블 데이터 분포 : 
 hazelnut-good                 313
screw-good                    256
carpet-good                   224
pill-good                     214
grid-good                     211
wood-good                     198
leather-good                  196
zipper-good                   192
tile-good                     184
cable-good                    179
metal_nut-good                176
capsule-good                  175
transistor-good               170
bottle-good                   167
toothbrush-good                48
toothbrush-defective           12
metal_nut-bent                 11
pill-crack                     11
screw-scratch_neck             11
screw-thread_side              10
pill-color                     10
screw-thread_top               10
metal_nut-flip                 10
screw-manipulated_front         9
capsule-poke                    9
bottle-contamination            9
capsule-scratc

3422
855
학습 레이블 데이터 분포 : 
 hazelnut-good                 313
screw-good                    256
carpet-good                   224
pill-good                     213
grid-good                     212
wood-good                     197
leather-good                  196
zipper-good                   192
tile-good                     184
cable-good                    179
capsule-good                  176
metal_nut-good                176
transistor-good               170
bottle-good                   167
toothbrush-good                48
toothbrush-defective           12
metal_nut-flip                 10
screw-scratch_head             10
pill-color                     10
capsule-scratch                10
metal_nut-scratch              10
metal_nut-bent                 10
screw-thread_top               10
screw-scratch_neck             10
capsule-crack                  10
screw-thread_side              10
screw-manipulated_front        10
pill-scratch                   10
pill-crack           

3422
855
학습 레이블 데이터 분포 : 
 hazelnut-good                 312
screw-good                    256
carpet-good                   224
pill-good                     214
grid-good                     211
wood-good                     198
leather-good                  196
zipper-good                   192
tile-good                     184
cable-good                    180
metal_nut-good                176
capsule-good                  175
transistor-good               171
bottle-good                   167
toothbrush-good                48
toothbrush-defective           12
pill-crack                     11
pill-color                     11
screw-scratch_neck             11
screw-manipulated_front        10
capsule-scratch                10
metal_nut-bent                 10
capsule-crack                  10
wood-scratch                    9
capsule-faulty_imprint          9
screw-scratch_head              9
bottle-broken_small             9
pill-contamination              9
bottle-contamination 

## StratifiedKFold

gc : Garbage Collector
gc.collect() : 메모리 관리 작업을 수행, 메모리 사용을 마쳤을 때 메모리를 비움

torch.cuda.empty_cache() : 사용되지 않는 GPU상 cache를 정리

torch.cuda.amp.autocast() : autocasting을 위한 모듈 제공
torch.cuda.amp.GradScaler() : 
    backward 시 underflow방지를 위해 float32범위로 스케일
    옵티마이저가 파라미터를 업데이트하기전에 원래 스케일대로 복구
    
optimizer.zero_grad() : 이상적인 학습을 위한 backward()를 호출할 때 하는 초기설정

eval() : train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    torch.no_grad()와 함께 쓰인다.
    evaluation(평가) 시 쓰지 말아야 할 layer들을 알아서 off 시키도록 하는 함수인 셈이다.
    
early_stopping : early_stoppying

In [ ]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 32
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), np.array(train_classes), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), np.array([]), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)


  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

#   optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
  optimizer = torch.optim.Adadelta(model.parameters())
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'0513_best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------
-----------------SAVE:1 epoch----------------
epoch : 1/70    time : 194s/13377s
TRAIN    loss : 1.06271    f1 : 0.19991
Val    loss : 0.53866    f1 : 0.33082
-----------------SAVE:2 epoch----------------
epoch : 2/70    time : 180s/12210s
TRAIN    loss : 0.50609    f1 : 0.37492
Val    loss : 0.39828    f1 : 0.44426
-----------------SAVE:3 epoch----------------
epoch : 3/70    time : 167s/11203s
TRAIN    loss : 0.37039    f1 : 0.56183
Val    loss : 0.28328    f1 : 0.58649
-----------------SAVE:4 epoch----------------
epoch : 4/70    time : 168s/11056s
TRAIN    loss : 0.30820    f1 : 0.59011
Val    loss : 0.21666    f1 : 0.61284
-----------------SAVE:5 epoch----------------
epoch : 5/70    time : 167s/10849s
TRAIN    loss : 0.23587    f1 : 0.69419
Val    loss : 0.22197    f1 : 0.73400
-----------------SAVE:6 epoch----------------
epoch : 6/70    time : 167s/10697s
TRAIN    loss : 0.19172    f1 : 0.74630
Val    loss : 0.11249    f1 : 0.79707
------------

In [10]:
pred_ensemble = []
batch_size = 32
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), np.array(train_classes), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'0513_best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
len(pred_ensemble)

5

In [ ]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv(path + "data/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [ ]:
submission.to_csv(path + "submission/b45F_norm_epoch70_4_2.csv", index = False)

## Efficientb4 모델 학습

사전 학습 모델의 성능 파악을 할 때 Fold 학습은 실행 시간이 오래걸려서 fold를 나누지 않은 데이터에 대해서 학습을 진행하고 성능을 비교하였습니다.

In [15]:
batch_size = 32
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), np.array(train_classes), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), np.array([]), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
# optimizer = torch.optim.Adadelta(model.parameters(), weight_decay = 1e-3)
optimizer = torch.optim.Adadelta(model.parameters())
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 32
epochs = 30

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b4_ra2_320-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_ra2_320-7eb33cd5.pth


### 추론

In [ ]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [ ]:
submission = pd.read_csv(path + "data/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [ ]:
# efficient b4, 30 epoch, adadelta, allflip
submission.to_csv(path + "submission/0512_b4_norm_30epoch_allflip.csv", index = False)